TCP/IP Introduction
===
The Internet is based on rules about how to make connections, exchange data, terminate connections, handle timeouts, and so on. These are called protocols, and they are arranged in layers.

The very lowest layer governs aspects such as electrical signals; each higher layer builds on those below. In the middle, more or less, is the IP (Internet Protocol) layer, which specifies how network locations are addressed and how packets (chunks) of data flow. In the layer above that, two protocols describe how to move bytes between locations:

- UDP (User Datagram Protocol):
This is used for short exchanges. A datagram is a tiny message sent in a single burst, like a note on a postcard.
- TCP (Transmission Control Protocol):
This protocol is used for longer-lived connections. It sends streams of bytes and ensures that they arrive in order without duplication.

Note
---
To distinguish the different computer devices on the internet, each object is assigned a *unique* ip. But Your local machine always has the IP address <font color="brown">127.0.0.1</font> and the name <font color="brown">localhost</font>. You might see this called the <font color="brown">loopback interface</font>. 

In [3]:
# readout the network card setting
!ifconfig -a >netconfig.txt

The first three lines are listed as follows:

```
lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 16384
	options=1203<RXCSUM,TXCSUM,TXSTATUS,SW_TIMESTAMP>
	inet 127.0.0.1 netmask 0xff000000 
```    

Most of the Internet with which we interact—the Web, database servers, and so on—is based on the TCP protocol running atop the IP protocol; so called, **TCP/IP**.

The lowest level of network programming uses a **socket**, which  coding is tedious. We will introduce it by examples. 

UDP Example
===
The client sends a string in a UDP datagram to a server, and the server returns a packet of data containing a string. 

1. udp server code

```python
from datetime import datetime
import socket

# test on one's box with 6789 port
server_address = ('localhost', 6789)
max_size = 4096

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(server_address)

data, client = server.recvfrom(max_size)

print('At', datetime.now(), client, 'said', data)
server.sendto(b'Are you talking to me?', client)
server.close()
```

The server has to set up networking through two methods:
- the first method, socket.socket, creates a socket, 
- and the second, bind, binds to it (listens to any data arriving at that IP address and port). 

`AF_INET` means we’ll create an Internet (IP) socket and `SOCK_DGRAM` means we’ll send and receive datagrams, i.e. use UDP.

At this point, the server sits and waits for a datagram to come in (recvfrom). When one arrives, the server wakes up and gets both the data and information about the client. The client variable contains the address and port combination needed to reach the client. The server ends by sending a reply and closing its connection.

**2.** udp_client codes

```python
import socket
from datetime import datetime

server_address = ('localhost', 6789)
max_size = 4096

print('Starting the client at', datetime.now())
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.sendto(b'Hey!', server_address)
data, server = client.recvfrom(max_size)
print('At', datetime.now(), server, 'said', data)
client.close()
```

Result
---
<br>
shell &gt; python udp_server.py&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
shell &gt; python udp_client.py<br>
<br>
Starting the server at 2016-11-03
21:04:07.695159&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Starting the client at 2016-11-03 21:04:56.374011<br>
Waiting for a client to call.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
At 2016-11-03 21:04:56.376209 ('127.0.0.1', 6789) said b'Are you
talking to me?'<br>
<font color="#c0c0c0">At 2016-11-03 21:04:56.376075 ('127.0.0.1',
59037) said b'Hey!'</font><br>

TCP Example
===
TCP is used for longer-lived connections, such as the Web. TCP delivers data in the order in which you send it. If there were any problems, it tries to send it again. Let’s shoot a few packets from client to server and back with TCP.

1. client code

```python

import socket
from datetime import datetime

address = ('localhost', 6789)
max_size = 1000

print('Starting the client at', datetime.now())
#  create socket object with AF_INET (use IPv4 addrss or hostname), SOCK_STREAM (TCP client)
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# connect
client.connect(address)
# send data
client.sendall(b'Hey!')
# recieve data
data = client.recv(max_size)
print('At', datetime.now(), 'someone replied', data)
# disconnect
client.close()
```

**2.** TCP server codes

```python
from datetime import datetime
import socket

address = ('localhost', 6789)
max_size = 1000

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# pass in the IP address and port we want the server to listen on
server.bind(address)
# start listening
server.listen(5)

# while  client connected, we receive the client socket into the client variable, 
# and the remote connection details into the addr variable
client, addr = server.accept()
data = client.recv(max_size)

print('At', datetime.now(), client, 'said', data)
client.sendall(b'Are you talking to me?')
client.close()
server.close()
```


Result
---
shell &gt; python
tcp_server.py&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
shell &gt;<font color="#cc9933"> python tcp_client.py</font><br>
&nbsp;<br>
Starting the server at 2016-11-03
21:21:07.515420&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp; &nbsp; <font color="#cc9933">Starting the client at
2016-11-03 21:21:33.648436</font><br>
Waiting for a client to
call.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp; &nbsp;&nbsp;&nbsp; <font color="#cc9933">At 2016-11-03
21:21:33.650712 someone replied b'Are you talking to me?'</font><br>
<font color="#c0c0c0">At 2016-11-03 21:21:33.650565
&lt;socket.socket fd=6, family=AddressFamily.AF_INET,
type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 6789),
raddr=('127.0.0.1', 49211)&gt; said b'Hey!'</font>

Notice
---
The TCP server called `client.sendall()` to respond, and the earlier UDP server called `client.sendto()`. TCP maintains the client-server connection across multiple socket calls and remembers the client’s IP address.
-  UDP sends messages, but their size is limited, and they’re not guaranteed to reach their destination.
-  TCP sends streams of bytes, not messages. You don’t know how many bytes the system will send or receive with each call.
- To exchange entire messages with TCP, you need some extra information to reas‐ semble the full message from its segments: a fixed message size (bytes), or the size of the full message, or some delimiting character.
- Because messages are bytes, not Unicode text strings, you need to use the Python bytes type.

ZeroMQ (ZMQ)
---
- Exchange entire messages
- Retry connections
- Buffer data to preserve it when the timing between senders and receivers doesn’t line up

In [14]:
import PyPDF2
book='../../../../Downloads//BlackHatPython.pdf'
pdfFileObj = open(book, 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)


In [15]:
pdfReader.numPages

195

In [19]:
pageObj = pdfReader.getPage(32)
pageObj.extractText()

'The Network: Basics\n   11client. We then connect the client to the server \n and send it some data\n . The last step is to receive some data back and print out the response \n. This is the simplest form of a TCP client, but the one you will write most \noften.\nIn the above code snippet, we are making some serious assumptions \nabout sockets that you de˜nitely want to be aware of. The ˜rst assump\n-tion is that our connection will always succeed, and the second is that the \n\nserver is always expecting us to send data ˜rst (as opposed to servers that \n\nexpect to send data to you ˜rst and await your response). Our third assump\n-tion is that the server will always send us data back in a timely fashion. We \n\nmake these assumptions largely for simplicity™s sake. While programmers \n\nhave varied opinions about how to deal with blocking sockets, exception-\n\nhandling in sockets, and the like, it™s quite rare for pentesters to build these \n\nniceties into the quick-and-dirty tools 